In [2]:
import pandas as pd

loan_df = pd.read_csv(r'C:\\Users\\grish\\OneDrive\\Desktop\\speciess_recognier\\navins_project\\LoanApprovalPrediction.csv')

# Display the first few rows and summary information about the dataset
loan_df.head()
loan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            598 non-null    object 
 1   Gender             598 non-null    object 
 2   Married            598 non-null    object 
 3   Dependents         586 non-null    float64
 4   Education          598 non-null    object 
 5   Self_Employed      598 non-null    object 
 6   ApplicantIncome    598 non-null    int64  
 7   CoapplicantIncome  598 non-null    float64
 8   LoanAmount         577 non-null    float64
 9   Loan_Amount_Term   584 non-null    float64
 10  Credit_History     549 non-null    float64
 11  Property_Area      598 non-null    object 
 12  Loan_Status        598 non-null    object 
dtypes: float64(5), int64(1), object(7)
memory usage: 60.9+ KB


In [3]:
# from google.colab import files
# uploaded = files.upload()

import pandas as pd

# Load the CSV file to check its contents
loan_data = pd.read_csv('LoanApprovalPrediction.csv')

# Display the first few rows of the dataset to understand its structure
loan_data.head(), loan_data.columns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Handling missing values
# Fill missing numerical values (LoanAmount and Loan_Amount_Term) with the median
imputer = SimpleImputer(strategy='median')
loan_data['LoanAmount'] = imputer.fit_transform(loan_data[['LoanAmount']])
loan_data['Loan_Amount_Term'] = imputer.fit_transform(loan_data[['Loan_Amount_Term']])

# Fill missing categorical values (Gender, Married, Dependents, Self_Employed) with the most frequent value
categorical_cols = ['Gender', 'Married', 'Dependents', 'Self_Employed', 'Credit_History']
imputer_cat = SimpleImputer(strategy='most_frequent')
loan_data[categorical_cols] = imputer_cat.fit_transform(loan_data[categorical_cols])

# Convert categorical variables to numerical values using Label Encoding
le = LabelEncoder()
loan_data['Gender'] = le.fit_transform(loan_data['Gender'])
loan_data['Married'] = le.fit_transform(loan_data['Married'])
loan_data['Education'] = le.fit_transform(loan_data['Education'])
loan_data['Self_Employed'] = le.fit_transform(loan_data['Self_Employed'])
loan_data['Property_Area'] = le.fit_transform(loan_data['Property_Area'])
loan_data['Loan_Status'] = le.fit_transform(loan_data['Loan_Status'])  # Target variable

# Splitting the data into features (X) and target (y)
X = loan_data.drop(columns=['Loan_ID', 'Loan_Status'])
y = loan_data['Loan_Status']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Model Building
# Using RandomForestClassifier for loan approval prediction
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Step 3: Model Evaluation
# Predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

accuracy, classification_rep


!pip install imblearn

from imblearn.over_sampling import SMOTE

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Retrain the RandomForest model on the balanced dataset
model_balanced = RandomForestClassifier(random_state=42)
model_balanced.fit(X_train_balanced, y_train_balanced)

# Predictions on the test set
y_pred_balanced = model_balanced.predict(X_test)

# Evaluate the balanced model
accuracy_balanced = accuracy_score(y_test, y_pred_balanced)
classification_rep_balanced = classification_report(y_test, y_pred_balanced)

accuracy_balanced, classification_rep_balanced

from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'class_weight': ['balanced', {0: 2, 1: 1}, {0: 3, 1: 1}]  # Emphasizing class 0
}

# Initialize RandomForest with GridSearchCV
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, scoring='accuracy', n_jobs=-1)

# Fit the model with GridSearchCV
grid_search.fit(X_train, y_train)

# Best parameters from grid search
best_params = grid_search.best_params_

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
y_pred_tuned = best_model.predict(X_test)

# Get the accuracy and classification report
accuracy_tuned = accuracy_score(y_test, y_pred_tuned)
classification_rep_tuned = classification_report(y_test, y_pred_tuned)

best_params, accuracy_tuned, classification_rep_tuned


({'class_weight': {0: 2, 1: 1}, 'max_depth': 10, 'n_estimators': 100},
 0.7916666666666666,
 '              precision    recall  f1-score   support\n\n           0       0.75      0.43      0.55        35\n           1       0.80      0.94      0.86        85\n\n    accuracy                           0.79       120\n   macro avg       0.78      0.68      0.71       120\nweighted avg       0.79      0.79      0.77       120\n')

In [4]:
import pickle

# Save the trained model to a file using pickle
with open('loan_approval_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)
